In [1]:
import requests 
import time 
import torch
import re
import pandas as pd
from datasets import Dataset
from bs4 import BeautifulSoup
import datetime
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter ,
    Language 
)

In [2]:
f = (datetime.datetime.now() - datetime.timedelta(days=29)).strftime("%Y-%m-%d")
to = datetime.datetime.now().strftime("%Y-%m-%d")
print(f, to)

2024-05-09 2024-06-07


## Function to remove tags and clear the document : 

In [3]:
def remove_tags(input_text):
    clean_text = re.sub('<.*?>', '', input_text)
    timing_pattern = r'\b\d{1,2}\.\d{2}[apmAPM]+\s+[GMTgmt]+\b' 
    clean_text = re.sub(timing_pattern, '', clean_text) 
    clean_text = re.sub(r'\s+', ' ', clean_text)
    clean_text = clean_text.replace('\n', ' ')
    pattern = r'© \d+ BBC\. The BBC is not responsible for the content of external sites\. Read about our approach to external linking\.'
    clean_text = re.sub(pattern, '', clean_text)
    # clean_text = re.sub(r' - .*', '', clean_text)

    return clean_text

<h1> Collect documents from APIs : <h1>

In [5]:
def scrape_article_engadget(url):
    try:
        response = requests.get(url)
        response.raise_for_status() 
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        section_element = soup.find('div', class_='caas-body-content')
    
        if not section_element:
            return None
       
        article_text = ''
        for p_tag in section_element.find_all('p'):
        # Remove HTML tags and leading/trailing whitespaces
            article_text += p_tag.get_text(strip=True) + '\n'

        # Return cleaned text (optional: additional cleaning steps here)
        return article_text
    except requests.exceptions.RequestException as e:
        print(f"Error: An error occurred while fetching the webpage: {e}")
        return None
    except Exception as e:
        print(f"Error: An unexpected error occurred: {e}")
        return None

scrape_article_engadget(f'https://newsapi.org/v2/everything?domains=engadget.com&from={f}&to={to}&apiKey=00664c93e32e4db8ae93f46a260d5fd1')

In [9]:
def fetch_engadget_news():
        to = datetime.datetime.now().strftime('%Y-%m-%d')
        url=f'https://newsapi.org/v2/everything?domains=engadget.com&from={f}&to={to}&apiKey=00664c93e32e4db8ae93f46a260d5fd1'
        Engadget_articles = []
        publish_dates=set()
        response = requests.get(url)
        print(response.status_code)
        if response.status_code == 200:
            data = response.json()
            articles = data.get('articles', [])
            print(articles)
            for article in articles:
                title = article['title']
                content = scrape_article_engadget(article['url'])
                if content is None:
                    break
                content = remove_tags(content)
                publish_date=article['publishedAt']
                if publish_date not in publish_dates:
                    publish_dates.add(publish_date)
                    Engadget_articles.append({'title': title, 'content': content, 'publishdate':publish_date})
        else:
            print(f"Number of total articles retrieved from fortune news source is : {len(Engadget_articles)}")
        return Engadget_articles

In [10]:
fetch_engadget_news()

200
[{'source': {'id': None, 'name': '[Removed]'}, 'author': None, 'title': '[Removed]', 'description': '[Removed]', 'url': 'https://removed.com', 'urlToImage': None, 'publishedAt': '1970-01-01T00:00:00Z', 'content': '[Removed]'}, {'source': {'id': None, 'name': '[Removed]'}, 'author': None, 'title': '[Removed]', 'description': '[Removed]', 'url': 'https://removed.com', 'urlToImage': None, 'publishedAt': '1970-01-01T00:00:00Z', 'content': '[Removed]'}, {'source': {'id': None, 'name': '[Removed]'}, 'author': None, 'title': '[Removed]', 'description': '[Removed]', 'url': 'https://removed.com', 'urlToImage': None, 'publishedAt': '1970-01-01T00:00:00Z', 'content': '[Removed]'}, {'source': {'id': None, 'name': '[Removed]'}, 'author': None, 'title': '[Removed]', 'description': '[Removed]', 'url': 'https://removed.com', 'urlToImage': None, 'publishedAt': '1970-01-01T00:00:00Z', 'content': '[Removed]'}, {'source': {'id': None, 'name': '[Removed]'}, 'author': None, 'title': '[Removed]', 'descri

[]

<h1>CNN</h1>

In [16]:
def scrape_article_content_sky(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        articles = soup.find('div', class_='section-wrap')
        if articles:
            body = articles.find('div', class_='sdc-article-body sdc-article-body--lead')
            # print(body)
            paragraphs = body.find_all('p')
            article_text = ''.join([p.text for p in paragraphs])
            return article_text
        else:
            return None


print(scrape_article_content_sky('https://www.skysports.com/football/transfer-paper-talk/12709/13149137/senior-england-player-approached-gareth-southgate-over-jack-grealish-squad-omission-paper-talk'))

The top stories and transfer rumours from Friday's newspapers...DAILY TELEGRAPHJack Grealish's omission from the England squad for Euro 2024 was such a surprise that one senior member of the party approached Gareth Southgate to try and get an understanding of his decision.Please use Chrome browser for a more accessible video playerOne of Europe's youngest coaches, St Pauli's Fabian Hurzeler, has emerged as a shock leading candidate for the Brighton job.Neal Maupay appeared to taunt nemesis James Maddison over the midfielder's exclusion from England's European Championship squad in a picture of himself smiling in front of a dartboard holding three darts.Romelu Lukaku has opened the door to a Saudi Pro League move but also has interest from AC Milan, Napoli and Jose Mourinho for a reunion with his former manager at Fenerbahce.Any Napoli move for Romelu Lukaku would only happen if they manage to sell Nigeria striker Victor Osimhen first.
              Transfer Centre LIVE!
            
  

In [4]:
import datetime
import requests
import pandas as pd

CNN_ARTICLES = 1000

def get_sky_news():
    titles = []
    contents = []
    ids = []
    index = 0
    publish_dates = []
    f = (datetime.datetime.now() - datetime.timedelta(days=15)).strftime('%Y-%m-%d')
    to = datetime.datetime.now().strftime('%Y-%m-%d')
    url = ( 'https://newsapi.org/v2/everything?'
            'domains=skysports.com&'
            'language=en&'
            f'from={f}&'
            f'to={to}&'
            'sortBy=publishedAt&'
            'pageSize=100&'
            'apiKey=00664c93e32e4db8ae93f46a260d5fd1')
    cnn_articles = []
    response = requests.get(url)
    data = response.json()
    print(data)
    if 'totalResults' in data:
        print(f'Total Articles retrieved from CNN: {data["totalResults"]}')
    else:
        print("Error: Total Results not found in response")
        return [], []

    if response.status_code == 200:
        articles = data['articles']
        for article in articles:
            title = article['title']
            url = article['url']
            # content = scrape_article_content_cnn(url)  # You need to implement this function
            # For now, let's just leave it as an empty string
            content = ""
            publish_date = article['publishedAt']
            publish_dates.append(publish_date)
            titles.append(title)
            ids.append(index)
            index += 1
            if content:
                content = remove_tags(content)
                contents.append(content)
                cnn_articles.append({'Title': title, 'Content': content})
    return publish_dates, titles

get_sky_news()

{'status': 'ok', 'totalResults': 244, 'articles': [{'source': {'id': None, 'name': 'Sky Sports'}, 'author': 'Ron Walker', 'title': 'Woeful England sign off for Euros with limp Iceland defeat', 'description': 'Woeful England lost their final warm-up game ahead of a tournament for the first time since 1968 as minnows Iceland left Wembley with a deserved 1-0 win.', 'url': 'https://www.skysports.com/football/news/11095/13149144/england-0-1-iceland-jon-dagur-thorsteinsson-nets-winner-as-minnows-stun-three-lions-in-final-euro-2024-warm-up', 'urlToImage': 'https://e0.365dm.com/24/06/1600x900/skysports-harry-kane-england_6575886.jpg?20240607202818', 'publishedAt': '2024-06-06T22:13:00Z', 'content': 'Woeful England lost their final warm-up game ahead of a tournament for the first time since 1968 as minnows Iceland left Wembley with a deserved 1-0 win.\r\nThe Three Lions were soundly beaten by the m… [+1369 chars]'}, {'source': {'id': None, 'name': 'Sky Sports'}, 'author': None, 'title': 'Papers

(['2024-06-06T22:13:00Z',
  '2024-06-06T21:30:00Z',
  '2024-06-06T18:34:00Z',
  '2024-06-06T17:00:00Z',
  '2024-06-06T16:30:00Z',
  '2024-06-06T16:04:00Z',
  '2024-06-06T15:05:00Z',
  '2024-06-06T15:03:00Z',
  '2024-06-06T15:03:00Z',
  '2024-06-06T13:00:00Z',
  '2024-06-06T12:49:00Z',
  '2024-06-06T12:05:00Z',
  '2024-06-06T11:00:00Z',
  '2024-06-06T11:00:00Z',
  '2024-06-06T10:00:00Z',
  '2024-06-06T09:45:00Z',
  '2024-06-06T09:00:00Z',
  '2024-06-06T08:50:00Z',
  '2024-06-06T08:22:00Z',
  '2024-06-06T08:00:00Z',
  '2024-06-06T08:00:00Z',
  '2024-06-06T07:00:00Z',
  '2024-06-06T06:54:00Z',
  '2024-06-06T06:00:00Z',
  '2024-06-05T21:35:00Z',
  '2024-06-05T17:50:00Z',
  '2024-06-05T16:29:00Z',
  '2024-06-05T16:00:00Z',
  '2024-06-05T14:30:00Z',
  '2024-06-05T13:41:00Z',
  '2024-06-05T13:24:00Z',
  '2024-06-05T12:38:00Z',
  '2024-06-05T11:03:00Z',
  '2024-06-05T11:00:00Z',
  '2024-06-05T09:53:00Z',
  '2024-06-05T09:51:00Z',
  '2024-06-05T07:18:00Z',
  '2024-06-05T07:00:00Z',
  '2024-06-0

<h1>NEWS API</h1>

In [35]:
NUMBER_OF_ARTICLES = 100
def get_bbc_news():
    url = f'https://newsapi.org/v2/everything?sources=bbc-news&language=en&pageSize=100&apiKey=00664c93e32e4db8ae93f46a260d5fd1'
    bbc_articles = []
    titles = []
    contents = []
    ids = []
    while (len(bbc_articles) < NUMBER_OF_ARTICLES):
        if (len(bbc_articles)%10 == 0):
            print(f'Number of collected documents so far are : {len(bbc_articles)} ')
        response = requests.get(url)
        data = response.json()
        articles = data.get('articles', [])
        for index, article in enumerate(articles):
            title = article['title']
            content = get_article_content(article['url'])  # You need to define get_article_content()
            content = remove_tags(content)
            bbc_articles.append({'Title': title, 'Content': content})
            titles.append({'Title': title})
            contents.append(content)
            ids.append(str(index))
            time.sleep(1)
    return bbc_articles, contents, titles, ids


def get_article_content(article_url):
    response = requests.get(article_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Extract text from all <p> tags within the article body
    article_content = ' '.join([p.get_text() for p in soup.find_all('p')])
    return article_content

# collected_articles, docs, titles, ids = get_bbc_news()


<h1>New York Times</h1>

In [14]:
def fetch_cnbc_news():
    '''
    This function fetches finance news articles from the CNBC API
    '''
    ny_articles = []
    f = (datetime.datetime.now() - datetime.timedelta(days=4)).strftime('%Y-%m-%d')
    to = datetime.datetime.now().strftime('%Y-%m-%d')
    url=f'https://newsapi.org/v2/everything?sources=daily-mail&from={f}&to={to}&apiKey=00664c93e32e4db8ae93f46a260d5fd1'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Error: ", response.status_code)
        return None

fetch_cnbc_news()

Error:  400


<h1>AlphaVantage API </h1>

In [36]:
NUMBER_OF_ARTICLES = 25
def fetch_and_store_data():
    url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=FOREX:USD&apikey=SPIKC6LBBQ3VUG8C'
    finance_news = []
    index = 0
    ids = []
    titles = []
    summaries = []
    while (len(finance_news) < NUMBER_OF_ARTICLES):
        r = requests.get(url)
        data = r.json()
        feeds = data['feed']
        for feed in feeds:
            titles.append(feed['title'])
            summaries.append(remove_tags(feed['summary']))
            ids.append(index)
            index += 1
        time.sleep(1)
    finance_news.append({'id': ids, 'title': titles, 'summary': summaries})
    df = pd.DataFrame(finance_news[0])
    return df
# news_df = fetch_and_store_data()
# news_df.head()

<h1> The Guardian API </h1>

In [37]:
api_key = 'd69e575e-91d0-45c1-97f4-ac0fe69f9899'

endpoint = 'https://content.guardianapis.com/search'
params = {
    'api-key': api_key,
    'show-fields': 'headline,body',
    'order-by': 'newest',
    'page-size': 200  # Number of articles to retrieve per request
}

In [38]:
NUMBER_OF_DOCS =8000
def Guardian_data():
    collected_articles = []
    docs =[] 
    titles=[]
    ids =[]
    while (len(collected_articles)<NUMBER_OF_DOCS):
        if(len(collected_articles)%10 == 0): 
            print(f"Number of collected documents so far are : {len(collected_articles)} ....")
        response = requests.get(endpoint, params)
        data = response.json() # data is a dictionary with keys 'response' and 'total' and 'currentPage' and 'pages' and 'results' and 'startIndex' and 'pageSize' and 'status' and 'userTier' and 'total'
        articles = data['response']['results']
        index=0
        for article in articles:
            Title = article['fields']['headline']
            Content = remove_tags(article['fields']['body'])
            collected_articles.append({'Title': Title, 'Content': Content})
            docs.append(Content)
            titles.append({"Title":Title})
            ids.append(str(index))
            index += 1
        time.sleep(10)
    return collected_articles, docs, titles, ids

# collected_articles, docs, titles, ids = Guardian_data()
# print(len(collected_articles)) # number of articles

In [39]:
data_dict = {
    "Title": titles,
    "Content": docs,
}

# Convert the dictionary to a Dataset
dataset = Dataset.from_dict(data_dict)
print(dataset)

ArrowInvalid: Column 1 named Content expected length 73 but got length 29

## RecursiveCharacterTextSplitter :

In [ ]:
document = collected_articles[3]['Content']
print("document: \n" ,document)
splitter =RecursiveCharacterTextSplitter.from_language(
    language =Language.PYTHON ,chunk_size=512 , chunk_overlap=100
)
chuncked_doc = splitter.create_documents([document])
print("chuncked document :\n", chuncked_doc[0])

KeyError: 'Content'

## TiktokenTextSplitter

In [ ]:
from semantic_text_splitter import TiktokenTextSplitter

# Maximum number of tokens in a chunk
max_tokens = 512
# Optionally can also have the splitter not trim whitespace for you
splitter = TiktokenTextSplitter("gpt-3.5-turbo", trim_chunks=False)

chunks = splitter.chunks(document, max_tokens)

for chunk in enumerate(chunks): 
    print("chunk : ", chunk)

chunk :  (0, ' The latest growth figures are slightly more positive than expected by the OBR in the autumn, for this year and the next two. But it’s still a weak growth outlook by long-term standards. Hunt says the OBR are forecasting: 2024: 0.8% growth, up from 0.7% growth expected in November. 2025: 1.9% growth, up from 1.4% growth expected in November. 2026: 2.2% growth, up from 2% growth expected in November. 2027: 1.8% growth, down from 2% growth expected in November. 2028: 1.7% growth, matching the 1.7% growth expected in November. Hunt is still talking about particular regional investments. His list includes measures for Cambridge, to fund a development corporation there. There will be money for transport there. And there is money for north Wales, including money to fund the renovation of Theatre Clwyd. And the government will be purchasing the Wylfa site on Anglesey, where there are plans for a nuclear power station. Hunt is now talking about measures to address historic underi

## HuggingFaceTextSplitter

In [ ]:
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer

# Maximum number of tokens in a chunk
max_tokens = 512
tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
splitter = HuggingFaceTextSplitter(tokenizer, trim_chunks=False)

chunks = splitter.chunks(document, max_tokens)
for chunk in enumerate(chunks): 
    print("chunk : ", chunk)

chunk :  (0, ' The latest growth figures are slightly more positive than expected by the OBR in the autumn, for this year and the next two. But it’s still a weak growth outlook by long-term standards. Hunt says the OBR are forecasting: 2024: 0.8% growth, up from 0.7% growth expected in November. 2025: 1.9% growth, up from 1.4% growth expected in November. 2026: 2.2% growth, up from 2% growth expected in November. 2027: 1.8% growth, down from 2% growth expected in November. 2028: 1.7% growth, matching the 1.7% growth expected in November. Hunt is still talking about particular regional investments. His list includes measures for Cambridge, to fund a development corporation there. There will be money for transport there. And there is money for north Wales, including money to fund the renovation of Theatre Clwyd. And the government will be purchasing the Wylfa site on Anglesey, where there are plans for a nuclear power station. Hunt is now talking about measures to address historic underi

# Split articles into chunks :

In [ ]:
chunked_articles =[]
chunked_titles = []
for i in range(len(collected_articles)):  
    article = collected_articles[i]['Content']
    title =collected_articles[i]['Title']
    max_tokens = 512 
    splitter = TiktokenTextSplitter("gpt-3.5-turbo", trim_chunks=False)
    chunks = splitter.chunks(article, max_tokens)
    if i%100 ==0: 
        print(f"Number of articles processed so far is : {i} ...")
    for chunk in chunks:
        chunked_articles.append(chunk)
        chunked_titles.append(title)
data_dict = {
    "Title": chunked_titles,
    "Content": chunked_articles,
}
dataset = Dataset.from_dict(data_dict)
dataset

Number of articles processed so far is : 0 ...
Number of articles processed so far is : 100 ...
Number of articles processed so far is : 200 ...
Number of articles processed so far is : 300 ...
Number of articles processed so far is : 400 ...
Number of articles processed so far is : 500 ...
Number of articles processed so far is : 600 ...
Number of articles processed so far is : 700 ...
Number of articles processed so far is : 800 ...
Number of articles processed so far is : 900 ...
Number of articles processed so far is : 1000 ...
Number of articles processed so far is : 1100 ...
Number of articles processed so far is : 1200 ...
Number of articles processed so far is : 1300 ...
Number of articles processed so far is : 1400 ...
Number of articles processed so far is : 1500 ...
Number of articles processed so far is : 1600 ...
Number of articles processed so far is : 1700 ...
Number of articles processed so far is : 1800 ...
Number of articles processed so far is : 1900 ...
Number of ar

Dataset({
    features: ['Title', 'Content'],
    num_rows: 26509
})

# Embedding into vectors :

In [ ]:
from transformers import BertTokenizer , BertModel

tokenizer =BertTokenizer.from_pretrained('bert-base-uncased')
model =BertModel.from_pretrained('bert-base-uncased')

In [ ]:
def mean_pooling(model_output , attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded =(attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float())
    sum_embeddings=torch.sum(token_embeddings * input_mask_expanded , 1)
    sum_mask =torch.clamp(input_mask_expanded.sum(1) , min=1e-9)
    return sum_embeddings /sum_mask

In [ ]:
def embed_text(examples):
    inputs= tokenizer(
        examples["Content"],padding=True , truncation=True , return_tensors ="pt"
    )
    with torch.no_grad():
        model_output= model(**inputs)
    pooled_embeds = mean_pooling(model_output , inputs["attention_mask"])
    return {"embedding":pooled_embeds.cpu().numpy()}

In [ ]:
small_set = (
    dataset.shuffle(seed=42)
           .select(range(200))
           .map(embed_text , batched =True , batch_size =128)
)

Map: 100%|██████████| 8000/8000 [2:29:09<00:00,  1.12s/ examples]  


In [ ]:
import numpy as np

embedding_list = small_set[0]["embedding"]
embedding_array = np.array(embedding_list)
print(embedding_array.shape)

(768,)


In [ ]:
def check_similarity(embedding1, embedding2):
    # Ensure both embeddings are tensors
    embedding1 = torch.tensor(embedding1)
    embedding2 = torch.tensor(embedding2)
    
    # Reshape the embeddings
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)
    
    similarity = torch.cosine_similarity(embedding1, embedding2).item()
    return similarity


def eliminate_duplicates(embeddings, threshold=1):
    unique_embeddings = [embeddings[0]]
    for emb in embeddings[1:]:
        similarities = [check_similarity(emb, unique_emb) for unique_emb in unique_embeddings]
        max_similarity = max(similarities, default=0)  # Default to 0 if no similarities found
        if max_similarity < threshold:
            unique_embeddings.append(emb)
        else:
            print("Duplicate found!")
    return unique_embeddings

from collections import defaultdict

def remove_duplicate_vectors(vectors):
    """
    This function removes duplicates embeddings from a list of embeddings

    Returns:
        A list containing unique embeddings
    """
    unique_vectors = []
    unique_vector_indices = defaultdict(list)
    for i, vector in enumerate(vectors):
        unique_vector_indices[tuple(vector)].append(i)
    for vector, indices in unique_vector_indices.items():
        unique_vectors.append(vector)
  
    return unique_vectors

unique_embeddings = remove_duplicate_vectors(embedding_array)

In [ ]:
#check if there are any duplicates in the embeddings to eliminate them
unique_embeddings = eliminate_duplicates(unique_embeddings, threshold=1)
small_set[0]["embedding"] = unique_embeddings

# News documents pretraitement  :

In [ ]:
NUMBER_OF_DOCS_COLLECTED =len(collected_articles)

n_rows =range(len(small_set))
small_set =small_set.add_column("idx" , n_rows)

NameError: name 'collected_articles' is not defined

In [ ]:
small_set

Dataset({
    features: ['Title', 'Content', 'embedding', 'idx'],
    num_rows: 8000
})

# Qdrant vectorial database : 

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import numpy as np
from qdrant_client.models import PointStruct
from datasets import load_dataset
from qdrant_client.http import models
from qdrant_client.http.models import CollectionStatus

c:\Users\rachi\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# QDRANT_KEY = 'NfNrrb963-CNc5SG7AjL0VbEEvBWCCNXsLXNIbtoNCiEQrUSeU61zA' 

In [2]:
client = QdrantClient(
    url="https://5c32ac64-b1f7-4665-91eb-e321a98c02f6.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="Wd_RTregmznFMCyDLagJHM_7a5TjJJuFLVTuMgfjQD44-BHLnhYbUg",
)

In [6]:
COLLECTION_NAME = 'News_test'
client.get_collection(collection_name=COLLECTION_NAME )

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=244, indexed_vectors_count=0, points_count=244, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={}

In [3]:
COLLECTION_NAME = 'News_test'
client.create_collection(collection_name=COLLECTION_NAME,vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE))

True

In [45]:
import uuid

myuuid = uuid.uuid4()
myuuidStr = str(myuuid)
print(myuuidStr)


myuuid = uuid.uuid4()
client.upsert(
    collection_name=COLLECTION_NAME,
    points=models.Batch(
        ids=[myuuidStr],  # Convert UUID to string
        payloads=[
            {"color": "red"},
        ],
        vectors=[
            [0.9, 0.1, 0.1],
        ],
    ),
)


b4e594ea-addf-444c-ae0f-0a1da0ebafe4


UpdateResult(operation_id=7, status=<UpdateStatus.COMPLETED: 'completed'>)

In [14]:
ids_to_delete = list(range(1, 547))
print("Deleting vectors with ids:", ids_to_delete)

client.delete(
    collection_name= COLLECTION_NAME,
    points_selector=models.PointIdsList(
        points= ids_to_delete,
    ),
)

Deleting vectors with ids: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 

UpdateResult(operation_id=19, status=<UpdateStatus.COMPLETED: 'completed'>)

In [64]:
COLLECTION_NAME = "News"
collection_info = client.get_collection(COLLECTION_NAME)
num_vectors = collection_info.points_count 
print("Number of vectors in the collection:", num_vectors)

Number of vectors in the collection: 12445


In [ ]:
def upload_documents_to_Qdrant(data, collection="news_articles"):
    # Get the current number of vectors in the collection
    collection_info = client.get_collection(collection)
    current_num_vectors = collection_info.vectors_count
    
    # Prepare points with payloads
    points = [
        models.PointStruct(
            id=new_id,
            vector=embedding,
            payload={
                "title": title,
                "content": content
            }
        )
        for new_id, (title, content, embedding) in zip(range(current_num_vectors + 1, current_num_vectors + 1 + len(data["embedding"])), zip(data["Title"], data["Content"], data["embedding"]))
    ]

    # Upload points to Qdrant
    client.upload_points(
        collection_name=collection,
        points=points
    )

    print("Uploaded:", len(data["embedding"]), "documents to the Qdrant database")

# Iterate through the dataset in batches
batch_size = 25
for i in range(0, len(small_set), batch_size):
    data = small_set[i:i+batch_size]
    upload_documents_to_Qdrant(data)

print(f"Successfully uploaded {len(small_set)} documents to the Qdrant database")


Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdrant database
Uploaded: 25 documents to the Qdra

In [ ]:
client.scroll(
    collection_name =COLLECTION_NAME , 
    limit =10 ,
    with_payload=False , 
    with_vectors=False
)

([Record(id=2, payload=None, vector=None, shard_key=None),
  Record(id=3, payload=None, vector=None, shard_key=None),
  Record(id=4, payload=None, vector=None, shard_key=None),
  Record(id=5, payload=None, vector=None, shard_key=None),
  Record(id=6, payload=None, vector=None, shard_key=None),
  Record(id=7, payload=None, vector=None, shard_key=None),
  Record(id=8, payload=None, vector=None, shard_key=None),
  Record(id=9, payload=None, vector=None, shard_key=None),
  Record(id=10, payload=None, vector=None, shard_key=None),
  Record(id=11, payload=None, vector=None, shard_key=None)],
 12)

In [ ]:
query=small_set[1]['embedding']
small_set[1]['Content'] , query[:7]

('We’re going to close our borders. We’re going to drill baby drill.” He has upended and inverted yet another political convention: optimism. Not for him Ronald Reagan’s morning in America or Bill Clinton’s place called Hope or Barack Obama’s yes, we can. Instead only murder, mayhem and total darkness. If only he had still been running things, he lamented, Russia would not have invaded Ukraine, Israel would not have been attacked and Iran would be broke. Now inflation is “destroying the middle class, it’s destroying everything”. “Our cities are being overrun with migrant crime, and that’s Biden migrant crime,” Trump grimaced. “But it’s a new category and it’s violent, where they’ll stand in the middle of the street and have fistfights with police officers. And if they did that in their countries from where they came, they’d be killed instantly. They wouldn’t do that. So the world is laughing at us. The world is taking advantage of us.” Read more here: Trump’s Super Tuesday victory spee

In [ ]:
result =client.search(
    collection_name=COLLECTION_NAME ,
    query_vector =query ,
    limit=1
)
result[0]

ScoredPoint(id=12836, version=3156, score=0.95170057, payload={'content': 'Tova S, a Los Angeles native said she was voting “as libertarian as possible” in all of the races on her ballot. At the national level, she’s most interested in Vivek Ramaswamy as a presidential candidate because of his stance on ending financial aid to other countries and locally she’s voting for Gail Lightfoot, a libertarian running for a US Senate seat. “They’re very out there, I look at them as the middle ground,” Tova said. “They give out-of-the-box thinking. I just don’t want the same old same old. I really want someone to make changes.” She also made it a point to research each candidate’s background, upbringing, and prior decisions to weed out those within whom she could have hope from those who were all about one-liners and photo ops. To her disappointment, she found that most candidates, whether they were running for a judgeship, senate seat, or position on the county school board, did not have a websi

In [ ]:
def embed_query(query_text):
    # Tokenize the query text
    query_inputs = tokenizer(
        query_text,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    
    # Embed the query
    with torch.no_grad():
        query_model_output = model(**query_inputs)
    
    query_embedding = mean_pooling(query_model_output, query_inputs["attention_mask"])
    
    return query_embedding

In [ ]:
query_embedding = embed_query("The Early Music Show, presented by Lucie Skeaping")

# Perform similarity search
query_result = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_embedding[0].tolist(),  # Convert tensor to list
    limit=5
)
